In [1]:
#imports
import pandas as pd
import csv
from csv import writer
from io import BytesIO

In [5]:
#load and reshape data
housing = pd.read_csv('../datasets/raw/housing/density_2010.csv', dtype={'GCT_STUB.target-geo-id2' : str })
water_systems = pd.read_csv('../datasets/raw/water/water_systems.csv')
violations = pd.read_csv('../datasets/raw/water/violations.csv', low_memory=False)
income = pd.read_csv('../datasets/transformed/county_hh_median_income.csv', dtype={'FIPS' : str}).set_index(['FIPS'])
geocoded_pws = pd.read_csv('../datasets/transformed/geocoded_public_water_systems.csv', dtype={ 'FIPS' : str })
violations['year'] = violations['Compliance Period Begin Date'].apply(lambda v : v[-4:])

In [7]:
#preparing data for look up

#violations
#group by year as to make getting counts easier
vio_year = violations.drop_duplicates()
vio_year = vio_year[['PWS ID', 'year']]
vio_year = vio_year.groupby(['PWS ID', 'year']).size().reset_index(name='counts')
vio_year = vio_year.set_index(['PWS ID', 'year'])

#housing
#make column names friendler
housing['FIPS'] = housing['GCT_STUB.target-geo-id2']
housing['housing_density'] = housing['SUBHD0402']
housing = housing.set_index('FIPS')

#water systems
#join them with their FIPS code
water_systems = water_systems.sort_values(by='PWS ID')
water_geo = pd.merge(water_systems, geocoded_pws[['PWS ID', 'FIPS']], on='PWS ID')


In [8]:
# The big join!
# columns in goal dataset : PWS_ID, year, total_violations, median_income, housing_density

years = list(map(lambda y : str(y), range(1982, 2016)))
csv_writer = None
with open('../datasets/final/cws_violations_p_year_w_demo.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['PWS_ID','year','total_violations','median_hh_income','housing_unit_density'])
    for pws in water_geo.itertuples():
        _id = getattr(pws, '_1')
        _fips = getattr(pws, 'FIPS')
        for i, _year in enumerate(years):
            _total_violations = 0
            _median_income = float('nan')
            _housing_density = float('nan')
            if (_id, _year) in vio_year.index:
                _total_violations = vio_year.loc[(_id, _year)]['counts']
            if (_fips) in income.index:
                _median_income = income.loc[(_fips)][_year]
            if (_fips) in housing.index:
                _housing_density = housing.loc[(_fips)]['housing_density']  
            row = [_id,_year,_total_violations,_median_income,_housing_density]
            csv_writer.writerow(row)
